In [16]:
import json
import pandas as pd
from utils import get_cds_range_lookup, intevl
from ribopy import Ribo
from matplotlib.ticker import PercentFormatter
import matplotlib.pyplot as plt
import numpy as np
import ribopy

In [17]:
study = 'GSE51584'
experiment = 'GSM1248729'
ribo_file =  f"/scratch/users/mjgeng/process-multiple-ribo/output/{study}/ribo/experiments/{experiment}.ribo"
ribo = Ribo(ribo_file, alias=ribopy.api.alias.apris_human_alias)
start, end, _ = intevl(ribo, experiment)
coverage = ribo.get_coverage(experiment, start, end, True)
boundary_lookup = get_cds_range_lookup(ribo)


In [18]:
data = json.load(open('data/dupe_idx_maps.json'))

In [19]:
correction_map_df = pd.DataFrame(data).map(len)
correction_map_df

,15,16,17,18,19,20,21,22,23,24,...,31,32,33,34,35,36,37,38,39,40
A1BG-201,274,116,46,22,8,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A1CF-202,267,88,40,22,15,11,8,5,3,1,...,0,0,0,0,0,0,0,0,0,0
A2M-201,1179,709,536,463,420,392,368,349,333,319,...,246,239,232,225,218,211,204,197,190,183
A2ML1-201,665,214,73,21,5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A3GALT2-201,199,76,25,7,3,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZXDC-202,611,275,133,77,50,38,29,21,14,11,...,0,0,0,0,0,0,0,0,0,0
ZYG11A-201,325,117,45,20,13,9,6,3,0,0,...,0,0,0,0,0,0,0,0,0,0
ZYG11B-201,319,102,29,8,4,3,2,1,0,0,...,0,0,0,0,0,0,0,0,0,0
ZYX-201,383,153,52,17,11,6,3,2,1,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
correction_map_df.to_csv("data/gene_correction_map.csv")

# Analysis

In [21]:
def get_correction(gene, range_lower, range_upper):
    return sum(correction_map_df.loc[gene][str(i)] for i in range(range_lower, range_upper + 1)) / (range_upper - range_lower + 1)

def get_readable_percent(gene, ribo):
    start, end = boundary_lookup[gene][1]
    intevl_start, intevl_end, _ = intevl(ribo, ribo.experiments[0])
    non_readable_percent = get_correction(gene, intevl_start, intevl_end) / (end - start)
    return max(1 - non_readable_percent, 0)

def get_readable_percent_rna(gene, ribo):
    start, end = boundary_lookup[gene][1]
    non_readable_percent = get_correction(gene, 40, 40) / (end - start)
    return max(1 - non_readable_percent, 0)

In [22]:
readable_percentages = []
for gene in coverage:
    readable_percentages.append(get_readable_percent(gene, ribo))

In [23]:
# percent of genes completely mapped
len([x for x in readable_percentages if x == 1]) / len(readable_percentages)

0.719041345764086

In [24]:
# percent of genes mostly mapped
len([x for x in readable_percentages if x > 0.95]) / len(readable_percentages)

0.8362383461694366

In [25]:
# percent of genes less than half mapped
len([x for x in readable_percentages if x < 0.5]) / len(readable_percentages)

0.09282529387920552

In [26]:
plt.close()
plt.hist(readable_percentages, bins=20, weights=np.ones(len(readable_percentages)) / len(readable_percentages))

plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
plt.savefig('readable_percentages_hist.png')

In [27]:
readable_percentages_rna = []
for gene in coverage:
    readable_percentages_rna.append(get_readable_percent_rna(gene, ribo))

In [28]:
plt.close()
plt.hist(readable_percentages_rna, bins=20, weights=np.ones(len(readable_percentages_rna)) / len(readable_percentages_rna))

plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
plt.savefig('readable_percentages_rna_hist.png')

In [29]:
diffs = []
for i, x in enumerate(readable_percentages_rna):
    diffs.append(x - readable_percentages[i])

In [30]:
plt.close()
plt.hist(diffs, bins=20, weights=np.ones(len(diffs)) / len(diffs))

plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
plt.savefig('readable_hist_rna_diff.png')